In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 808.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="",
    api_key = ""
)
def call_gpt4(messages):
  response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=32,
        temperature=0,
        seed=0,
        messages=message
  )
  return response

# 测试调用
messages = "解释一下量子力学的基本原理。"
response = call_gpt4(messages)
print(response.model)

gpt-35-turbo


In [ ]:
import argparse
import json
import re
import os
from tqdm import tqdm
import numpy as np
from openai import OpenAI

client = OpenAI(
  base_url="",
    api_key = ""
)
# 用于匹配if...then...句式的正则表达式
pattern = r'\bif\b.*\bthen\b'
pattern_or = r'\bnot\b.*\bor\b'
pattern_yes = r'\bYes\b'
pattern_not = r'\bnot\b.*\bnot\b'
pattern_unless = r'\bnot\b.*\bunless\b'

def judge_transitive(judgeMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Can the following sentence structures apply the Transitive Law:(α → β) ∧ (β → γ) ⇔ (α → γ)?. If you have no keyboarding skills at all, you will not be able to use a computer. And if you are not able to use a computer, you will not be able to write your essays using a word processing program."},
      {"role": "system", "content": "Yes, the sentences structures can apply the Transitive Law. If having no keyboarding skills (α) implies not being able to use a computer (β), and not being able to use a computer (β) implies not being able to write essays using a word processing program (γ), then having no keyboarding skills (α) implies not being able to write essays using a word processing program (γ)."},
      {"role": "user", "content": "Can the following sentence structures apply the Transitive Law:(α → β) ∧ (β → γ) ⇔ (α → γ)?. If a spoken language is completely efficient, then every possible permutation of its basic language sounds can be an understandable word. However, if the human auditory system is an imperfect receptor of sounds, then it is not true that every possible permutation of a spoken language' s basic language sounds can be an understandable word"},
      {"role": "system", "content": "No, the sentences cannot apply the Transitive Law because they do not form a direct logical chain. The first sentence discusses the efficiency of spoken language, but the second sentence does not directly connect to the first one. The second sentence introduces a new conditional statement about the human auditory system without directly relating to the first sentence. Therefore, there is no direct path to apply the Transitive Law."},
  ]
  conversation_history.append({"role": "user", "content": judgeMessage})
  return chat(conversation_history)
def judge_contraposition(judgeMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Can the following sentence structures apply the Contraposition Law:(A → B) ⇔ (¬B → ¬A)?. If Alan is kind, then Bob is clever."},
      {"role": "system", "content": "Yes, the contraposition law can be applied to the sentence 'If Alan is kind, then Bob is clever.' The contraposition of the statement 'If A, then B' is 'If not B, then not A."},
      {"role": "user", "content": "Can the following sentence structures apply theContraposition Law:(A → B) ⇔ (¬B → ¬A)?. If Bob is not clever, then Alan is not kind."},
      {"role": "system", "content": "Yes, the contraposition law can be applied to the sentence 'If Bob is not clever, then Alan is not kind.' Let's analyze it: Original statement: If Bob is not clever, then Alan is not kind.(¬B → ¬A) Contrapositive statement: If Alan is kind, then Bob is clever.(A → B) So, the contrapositive statement is logically equivalent to the original statement, satisfying the Contraposition Law."},
      {"role": "user", "content": "Can the following sentence structures apply theContraposition Law:(A → B) ⇔ (¬B → ¬A)?. The bald eagle isn’t small, unless the mouse is small."},
      {"role": "system", "content": "Yes, to determine if the sentence 'The bald eagle isn’t small, unless the mouse is small.' can apply the Contraposition Law, we need to rephrase the sentence in the form of an implication and then check its contrapositive.The sentence 'The bald eagle isn’t small, unless the mouse is small' can be interpreted as an implication:'Unless the mouse is small' means 'if the mouse is not small.'So the sentence can be rewritten as: 'If the mouse is not small, then the bald eagle isn’t small.'Let's define the statements more clearly:A: 'The mouse is small.'B: 'The bald eagle is small.'The sentence 'If the mouse is not small, then the bald eagle isn’t small' can be written as:¬A → ¬B Now we need to check its contrapositive:The contrapositive of ¬A → ¬B is B → A. In words, the contrapositive statement is: 'If the bald eagle is small, then the mouse is small.'So, the sentence 'The bald eagle isn’t small, unless the mouse is small.' (¬A → ¬B) is logically equivalent to 'If the bald eagle is small, then the mouse is small.' (B → A) by the Contraposition Law.Thus, the sentence can indeed apply the Contraposition Law."},

  ]
  conversation_history.append({"role": "user", "content": judgeMessage})
  return chat(conversation_history)

def judge_implication(judgeMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Can the following sentence structures apply the Implication Law:(A → B) ⇔ (¬A ∨ B)?. If Alan is kind, then Bob is clever."},
      {"role": "system", "content": "Yes, the implication law applies to the sentence 'If Alan is kind, then Bob is clever.' Let's break it down: Original statement: If Alan is kind, then Bob is clever. (A → B) Equivalent statement using the implication law: (¬A ∨ B) In the equivalent statement: ¬A represents 'Alan is not kind.' B represents 'Bob is clever.' So, according to the implication law, the original statement 'If Alan is kind, then Bob is clever' is logically equivalent to '(¬A ∨ B)'."},
      {"role": "user", "content": "Can the following sentence structures apply the Implication Law:(A → B) ⇔ (¬A ∨ B)?. Alan is not kind or Bob is clever."},
      {"role": "system", "content": "Yes, the implication law applies to the sentence 'Alan is not kind or Bob is clever.' Let's break it down: Original statement: Alan is not kind or Bob is clever. (¬A ∨ B) Equivalent statement using the implication law: (A → B) In the equivalent statement: A represents 'Alan is kind.' B represents 'Bob is clever.' So, according to the implication law, the original statement 'Alan is not kind or Bob is clever' is logically equivalent to '(A → B)'."},

  ]
  conversation_history.append({"role": "user", "content": judgeMessage})
  return chat(conversation_history)

def chat_contraposition(chatMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Apply Contraposition Law:(A → B) ⇔ (¬B → ¬A) to answer the question. If Alan is kind, then Bob is clever."},
      {"role": "system", "content": "If Bob is not clever, then Alan is not kind"},
      {"role": "user", "content": "Apply Contraposition Law:(A → B) ⇔ (¬B → ¬A) to answer the question. If Bob is not clever, then Alan is not kind."},
      {"role": "system", "content": "If Alan is kind, then Bob is clever."},
      {"role": "user", "content": "Apply Contraposition Law:(A → B) ⇔ (¬B → ¬A) to answer the question. The bald eagle isn’t small, unless the mouse is small."},
      {"role": "system", "content": "If the bald eagle is small, then the mouse is small."},

  ]
  conversation_history.append({"role": "user", "content": "Please strictly follow the above example to return the answer. Only return results without explanation."+chatMessage})
  return chat(conversation_history)

def chat_implication(chatMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Apply Implication Law:(A → B) ⇔ (¬A ∨ B) to answer the question. If Alan is kind, then Bob is clever."},
      {"role": "system", "content": "Alan is not kind or Bob is clever."},
      {"role": "user", "content": "Apply Implication Law:(A → B) ⇔ (¬A ∨ B) to answer the question. Alan is not kind or Bob is clever."},
      {"role": "system", "content": "If Alan is kind, then Bob is clever."},
  ]
  conversation_history.append({"role": "user", "content": "Please strictly follow the above example to return the answer. Only return results without explanation."+chatMessage})
  return chat(conversation_history)

def chat_transitive(chatMessage):
  # 初始化对话历史
  conversation_history = [
      {"role": "user", "content": "Apply Transitive Law:(α → β) ∧ (β → γ) ⇔ (α → γ) to answer the question. If you have no keyboarding skills at all, you will not be able to use a computer. And if you are not able to use a computer, you will not be able to write your essays using a word processing program."},
      {"role": "system", "content": "If you have no keyboarding skills at all, you will not be able to write your essays using a word processing program."},
  ]
  conversation_history.append({"role": "user", "content": "Please strictly follow the above example to return the answer. Only return results without explanation."+chatMessage})
  return chat(conversation_history)

def extend_option(option):
  if re.search(pattern, option.lower()):
      contraposition_message = chat_contraposition(option)
      implication_message = chat_implication(option)
      # print("option应用反对称律和蕴涵律")
      option = option + contraposition_message + implication_message
      # option = option + contraposition_message
  elif re.search(pattern_not, option.lower()):
      judge_message = judge_contraposition(option)
      if re.search(pattern_yes, judge_message):
        contraposition_message = chat_contraposition(option)
        # print("option应用反对称律")
        option = option + contraposition_message
  elif re.search(pattern_or, option.lower()):
      judge_message = judge_implication(option)
      if re.search(pattern_yes, judge_message):
        implication_message = chat_implication(option)
        # print("option应用蕴涵律")
        option = option + implication_message
  return option

def chat(messages):
  response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=32,
        temperature=0,
        seed=0,
        messages=messages
  )
  return response.choices[0].message.content


# 设置模型和结果文件名
model = "gpt-3.5-turbo"
results_file = "reclor_test_gpt-3.5_ours_results"

results_file_json = results_file + ".json"
if os.path.exists(results_file_json):
    with open(results_file_json, "r") as f:
        zero_shot_results = json.load(f)
else:
    zero_shot_results = []

with open("/content/drive/MyDrive/project/test.json", "r") as f:
    data = json.load(f)

start = len(zero_shot_results)
# 加载 JSON 文件
with open("/content/drive/MyDrive/project/test_predicted_results.json", 'r') as file:
    data_predict = json.load(file)

# 创建一个数组来存储匹配到的句子
matched_sentences = []

predicted_class = 666

ex_context = ""
ex_option1= ""
ex_option2= ""
ex_option3= ""
ex_option4= ""


for i in tqdm(range(start, len(data))):
    question = data[i]['question']
    key = str(i)
    # print("key----:"+key)
    question_type = data_predict[key]
    # print("question_type----:"+ str(question_type))
    context = data[i]['context']
    option1 = data[i]['answers'][0]
    option2 = data[i]['answers'][1]
    option3 = data[i]['answers'][2]
    option4 = data[i]['answers'][3]
    if question_type in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]:
      sentences = re.split(r'[.!?]', context)  # 使用句号、问号和感叹号来切分句子

      ex_context = context
      ex_option1 = option1
      ex_option2 = option2
      ex_option3 = option3
      ex_option4 = option4

      context_matched_sentences = []

      for sentence in sentences:
          if re.search(pattern, sentence.lower()):
              context_matched_sentences.append(sentence)
              contraposition_message = chat_contraposition(sentence)
              implication_message = chat_implication(sentence)
              # print("context反对称律和蕴涵律")
              ex_context = ex_context + contraposition_message + implication_message
              # ex_context = ex_context + contraposition_message
          if re.search(pattern_not, sentence.lower()):
              judge_message = judge_contraposition(sentence)
              if re.search(pattern_yes, judge_message):
                contraposition_message = chat_contraposition(sentence)
                # print("context应用反对称律")
                ex_context = ex_context + contraposition_message

          if re.search(pattern_unless, sentence.lower()):
              judge_message = judge_contraposition(sentence)
              if re.search(pattern_yes, judge_message):
                contraposition_message = chat_contraposition(sentence)
                # print("context应用反对称律")
                ex_context = ex_context + contraposition_message

          if re.search(pattern_or, sentence.lower()):
              judge_message = judge_implication(sentence)
              if re.search(pattern_yes, judge_message):
                implication_message = chat_implication(sentence)
                # print("context应用蕴含律")
                ex_context = ex_context + implication_message

      if len(context_matched_sentences) > 1:
          combined_string = ". ".join(context_matched_sentences)
          judge_message = judge_transitive("Can the following sentence structures apply the Transitive Law:(α → β) ∧ (β → γ) ⇔ (α → γ)?." + combined_string)
          if re.search(pattern_yes, judge_message):
              transitive_message = chat_transitive(combined_string)
              contraposition_message = chat_contraposition(transitive_message)
              implication_message = chat_implication(transitive_message)
              # print("context应用传递律")
              ex_context = ex_context + transitive_message +contraposition_message + implication_message



      ex_option1 = extend_option(option1)
      ex_option2 = extend_option(option2)
      ex_option3 = extend_option(option3)
      ex_option4 = extend_option(option4)
      final_message = ex_context + " " + question + \
          "\nA. " + ex_option1 + \
          "\nB. " + ex_option2 + \
          "\nC. " + ex_option3 + \
          "\nD. " + ex_option4 + \
          "\nAmong A to D, the answer is: "
      # print(final_message)
    else :
      final_message = context + " " + question + \
          "\nA. " + option1 + \
          "\nB. " + option2 + \
          "\nC. " + option3 + \
          "\nD. " + option4 + \
          "\nAmong A to D, the answer is: "
    messages = [
        {"role": "user", "content": final_message},
    ]

    response = client.chat.completions.create(
        model=model,
        max_tokens=32,
        temperature=0,
        seed=0,
        messages=messages)

    content = response.choices[0].message.content
    try:
        pred = re.findall(r'A|B|C|D', content)[0]
    except:
        try_time = 1
        while try_time < 5:
            response = client.chat.completions.create(
                model=model,
                max_tokens=32,
                temperature=1,
                messages=messages)
            content = response.choices[0].message.content
            pred = re.findall(r'A|B|C|D', content)
            if len(pred) > 0:
                pred = pred[0]
                break
            else:
                try_time += 1
                print(f"Try {try_time} times for {i}th question")

    results = json.loads(response.model_dump_json())
    results['pred'] = pred
    results['input'] = final_message
    results['question_type'] = question_type
    zero_shot_results.append(results)

    with open(results_file_json, "w") as f:
        json.dump(zero_shot_results, f, indent=4)

assert len(zero_shot_results) == len(data)
map_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
results_npy = []
for i in range(len(zero_shot_results)):
    results_npy.append(map_dict[zero_shot_results[i]['pred']])
print(results_npy)
print(len(results_npy))
results_npy = np.array(results_npy)
results_file_npy = results_file + ".npy"
np.save(results_file_npy, results_npy)

100%|██████████| 1000/1000 [15:33<00:00,  1.07it/s]

[3, 1, 2, 0, 1, 2, 1, 0, 2, 2, 1, 3, 3, 1, 2, 0, 3, 1, 1, 2, 2, 3, 0, 2, 0, 2, 2, 1, 3, 3, 1, 2, 1, 1, 1, 2, 1, 2, 3, 2, 3, 3, 1, 3, 1, 3, 3, 2, 1, 0, 3, 1, 2, 3, 2, 3, 3, 2, 1, 1, 0, 1, 1, 0, 3, 1, 3, 3, 0, 1, 3, 2, 0, 2, 2, 2, 0, 1, 1, 0, 3, 0, 1, 2, 2, 2, 0, 0, 2, 2, 1, 3, 0, 0, 1, 2, 3, 2, 0, 2, 2, 2, 2, 1, 1, 3, 0, 3, 1, 0, 3, 1, 2, 3, 3, 3, 2, 3, 0, 1, 2, 0, 3, 1, 0, 3, 2, 2, 2, 3, 3, 3, 1, 3, 2, 1, 0, 1, 3, 0, 3, 2, 2, 0, 0, 3, 1, 2, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1, 3, 1, 2, 3, 1, 2, 1, 3, 1, 2, 3, 0, 3, 2, 2, 0, 0, 3, 0, 2, 1, 1, 2, 2, 1, 2, 3, 3, 3, 1, 1, 1, 2, 0, 3, 2, 2, 0, 3, 1, 2, 0, 0, 2, 0, 2, 3, 3, 2, 0, 2, 3, 2, 3, 1, 3, 0, 2, 3, 3, 1, 1, 1, 3, 0, 3, 3, 1, 2, 0, 2, 3, 2, 1, 0, 3, 1, 3, 1, 0, 1, 0, 3, 2, 3, 1, 2, 0, 3, 0, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 1, 0, 1, 2, 1, 2, 1, 3, 1, 2, 0, 1, 3, 1, 0, 2, 3, 2, 0, 0, 1, 0, 3, 2, 0, 3, 0, 2, 3, 3, 0, 1, 1, 1, 3, 3, 1, 0, 0, 0, 3, 2, 2, 2, 1, 1, 3, 0, 1, 3, 1, 2, 3, 1, 0, 3, 1, 3, 3, 0, 3, 1, 2, 1, 2, 0, 0, 3, 3, 0, 1, 0, 